In [1]:
import numpy as np
import sympy as sym
import pydae.build_cffi as db

In [5]:


def dynph_conv(var1,var2,k_dict,k,verb = False):
    k_list = list(k_dict.keys())
    conv = 0
    conj_var2 = False
    if 'conj' in var2:
        var2 = var2.split('(')[1][:-1]
        conj_var2 = True
    conv = 0
    for l in range(-10,10):
        m = k - l
        #print(f'm = {m:3d}, l= {l:3d}')

        if m in k_list  or -m in k_list:
            if l in k_list or -l in k_list:
                sym1r,sym1i = sym.symbols(f'{var1}{k_dict[np.abs(m)]}r,{var1}{k_dict[np.abs(m)]}i', real=True)
                sym2r,sym2i = sym.symbols(f'{var2}{k_dict[np.abs(l)]}r,{var2}{k_dict[np.abs(l)]}i', real=True)
                sym_1 = sym1r + sym.I*sym1i
                if conj_var2:
                    sym_2 = sym.conjugate(sym2r + sym.I*sym2i)
                else:
                    sym_2 = (sym2r + sym.I*sym2i)
                        
                
                if m < 0: 
                    if verb: print(f'conj(x_({-m:2d})) * y_({l:2d})')
                    conv += sym.conjugate(sym_1)*sym_2
                elif l < 0:
                    if verb: print(f'x_({m:2d}) * conj(y_({-l:2d}))')
                    conv += sym_1*sym.conjugate(sym_2)
                else:
                    if verb: print(f'x_({m:2d})) * y_({l:2d})')
                    conv += sym_1*sym_2
    return conv

k = 2
k_list = [0,2]
k_dict = {0:'z',2:'n'}
var1 = 'x'
var2 = 'conj(y)'
dynph_conv(var1,var2,k_dict,k)

(I*xni + xnr)*(-I*yzi + yzr) + (I*xzi + xzr)*(-I*yni + ynr)

In [12]:
# https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=4538460&tag=1

omega,R,L = sym.symbols('omega,R,L', real=True) 

v_td0r,v_td0i,v_tq0r,v_tq0i = sym.symbols('v_td0r,v_td0i,v_tq0r,v_tq0i', real=True)
v_td2r,v_td2i,v_tq2r,v_tq2i = sym.symbols('v_td2r,v_td2i,v_tq2r,v_tq2i', real=True)

i_d0r,i_d0i,i_q0r,i_q0i = sym.symbols('i_d0r,i_d0i,i_q0r,i_q0i', real=True)
i_d2r,i_d2i,i_q2r,i_q2i = sym.symbols('i_d2r,i_d2i,i_q2r,i_q2i', real=True)

v_sd0r,v_sd0i,v_sq0r,v_sq0i = sym.symbols('v_sd0r,v_sd0i,v_sq0r,v_sq0i', real=True)
v_sd2r,v_sd2i,v_sq2r,v_sq2i = sym.symbols('v_sd2r,v_sd2i,v_sq2r,v_sq2i', real=True)


j = sym.I

v_td0 = v_td0r + j*v_td0i
v_tq0 = v_tq0r + j*v_tq0i
v_td2 = v_td2r + j*v_td2i
v_tq2 = v_tq2r + j*v_tq2i

i_d0 = i_d0r + j*i_d0i
i_q0 = i_q0r + j*i_q0i
i_d2 = i_d2r + j*i_d2i
i_q2 = i_q2r + j*i_q2i

v_sd0 = v_sd0r + j*v_sd0i
v_sq0 = v_sq0r + j*v_sq0i
v_sd2 = v_sd2r + j*v_sd2i
v_sq2 = v_sq2r + j*v_sq2i

# di_d = (v_d - R * i_d)/L
# di_q = (v_q - R * i_q)/L

#K = 0
di_d0 = 1/L*(v_td0 - R * i_d0 + L*omega*i_q0 - v_sd0) - j*( 0)*omega*i_d0
di_q0 = 1/L*(v_tq0 - R * i_q0 - L*omega*i_d0 - v_sq0) - j*( 0)*omega*i_q0

#K = 2
di_d2 = 1/L*(v_td2 - R * i_d2 + L*omega*i_q2 - v_sd2) - j*(+2)*omega*i_d2
di_q2 = 1/L*(v_tq2 - R * i_q2 - L*omega*i_d2 - v_sq2) - j*(+2)*omega*i_q2

k_dict = {0:'0',2:'2'}
s_abc_t0 = 1.5*(dynph_conv('v_td','conj(i_d)',k_dict,0) + dynph_conv('v_tq','conj(i_q)',k_dict,0))
s_abc_t2 = 1.5*(dynph_conv('v_td','conj(i_d)',k_dict,2) + dynph_conv('v_tq','conj(i_q)',k_dict,2))
s_abc_s0 = 1.5*(dynph_conv('v_sd','conj(i_d)',k_dict,0) + dynph_conv('v_sq','conj(i_q)',k_dict,0))
s_abc_s2 = 1.5*(dynph_conv('v_sd','conj(i_d)',k_dict,2) + dynph_conv('v_sq','conj(i_q)',k_dict,2))



#s_avg = v_n*conj(i_n) + v_0*conj(i_0) +v_p*conj(i_p) + v_2*conj(i_2)
#s_osc_n1 = v_n*conj(i_0) + v_0*conj(i_p) + v_p*conj(i_2)
#s_osc_n2 = v_n*conj(i_0) + v_0*conj(i_p) + v_p*conj(i_2)




#s_osc_n = v_n*conj(i_0) + v_0*conj(i_p) + v_p*conj(i_2) 

#s_osc_0 = v_0*conj(i_n) + v_0*conj(i_p) +v_0*conj(i_2) 

#v_t = v_n*exp(-jwt)

#s_abc_sn = 1.5*(dynph_conv('v_sd','i_d',k_dict,'n') + dynph_conv('v_sq','i_q',k_dict,'n'))
#s_abc_s0 = 1.5*(dynph_conv('v_sd','i_d',k_dict,'0') + dynph_conv('v_sq','i_q',k_dict,'0'))
#s_abc_sp = 1.5*(dynph_conv('v_sd','i_d',k_dict,'p') + dynph_conv('v_sq','i_q',k_dict,'p'))
#s_abc_s2 = 1.5*(dynph_conv('v_sd','i_d',k_dict,'2') + dynph_conv('v_sq','i_q',k_dict,'2'))

#s_s_abc = s_abc_sn + s_abc_s0 + s_abc_sp + s_abc_s2

params_dict = {'R':0.1,'L':0.01}


f_list = [sym.re(di_d0),sym.im(di_d0),
          sym.re(di_q0),sym.im(di_q0),
          sym.re(di_d2),sym.im(di_d2),
          sym.re(di_q2),sym.im(di_q2),
         ]
x_list = [i_d0r,i_d0i,
          i_q0r,i_q0i,
          i_d2r,i_d2i,
          i_q2r,i_q2i,
         ]
g_list = []
y_list = []

u_ini_dict = {'omega':2*np.pi*50,
'v_td0r':0.0,'v_tq0r':0.0,
'v_td2r':0.0,'v_tq2r':0.0,
'v_td0i':0.0,'v_tq0i':0.0,
'v_td2i':0.0,'v_tq2i':0.0,
'v_sd0r':0.0,'v_sq0r':0.0,
'v_sd2r':0.0,'v_sq2r':0.0,
'v_sd0i':0.0,'v_sq0i':0.0,
'v_sd2i':0.0,'v_sq2i':0.0,
}
u_run_dict = u_ini_dict


h_dict = {
'v_td0r':v_td0r,'v_tq0r':v_tq0r,
'v_td2r':v_td2r,'v_tq2r':v_tq2r,
'v_td0i':v_td0i,'v_tq0i':v_tq0i,
'v_td2i':v_td2i,'v_tq2i':v_tq2i,
'v_sd0r':v_sd0r,'v_sq0r':v_sq0r,
'v_sd2r':v_sd2r,'v_sq2r':v_sq2r,
'v_sd0i':v_sd0i,'v_sq0i':v_sq0i,
'v_sd2i':v_sd2i,'v_sq2i':v_sq2i,
's_abc_t0r':sym.re(s_abc_t0),'s_abc_t0i':sym.im(s_abc_t0),
's_abc_t2r':sym.re(s_abc_t2),'s_abc_t2i':sym.im(s_abc_t2),
's_abc_s0r':sym.re(s_abc_s0),'s_abc_s0i':sym.im(s_abc_s0),
's_abc_s2r':sym.re(s_abc_s2),'s_abc_s2i':sym.im(s_abc_s2),
}

In [13]:
s_abc_t0

1.5*(-I*i_d0i + i_d0r)*(I*v_td0i + v_td0r) + 1.5*(-I*i_d2i + i_d2r)*(-I*v_td2i + v_td2r) + 1.5*(I*i_d2i + i_d2r)*(I*v_td2i + v_td2r) + 1.5*(-I*i_q0i + i_q0r)*(I*v_tq0i + v_tq0r) + 1.5*(-I*i_q2i + i_q2r)*(-I*v_tq2i + v_tq2r) + 1.5*(I*i_q2i + i_q2r)*(I*v_tq2i + v_tq2r)

In [14]:
s_abc_s0

1.5*(-I*i_d0i + i_d0r)*(I*v_sd0i + v_sd0r) + 1.5*(-I*i_d2i + i_d2r)*(-I*v_sd2i + v_sd2r) + 1.5*(I*i_d2i + i_d2r)*(I*v_sd2i + v_sd2r) + 1.5*(-I*i_q0i + i_q0r)*(I*v_sq0i + v_sq0r) + 1.5*(-I*i_q2i + i_q2r)*(-I*v_sq2i + v_sq2r) + 1.5*(I*i_q2i + i_q2r)*(I*v_sq2i + v_sq2r)

In [15]:
sys = {'name':f"rl_dynph3",
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':h_dict
      }

bldr = db.builder(sys)
bldr.build()

system without algebraic equations, adding dummy algebraic equation
Compilation time: 3.15 s


In [68]:
sym.simplify(sym.re(s_abc_p))

-1.5*i_d0i*v_dpi + 1.5*i_d0r*v_dpr - 1.5*i_dpi*v_d0i + 1.5*i_dpr*v_d0r - 1.5*i_q0i*v_qpi + 1.5*i_q0r*v_qpr - 1.5*i_qpi*v_q0i + 1.5*i_qpr*v_q0r

In [27]:
sym.N(sym.re(s_abc_tp)/1.5)

-1.0*i_d0i*v_tdpi + 1.0*i_d0r*v_tdpr + 1.0*i_d2i*v_tdpi + 1.0*i_d2r*v_tdpr - 1.0*i_dpi*v_td0i + 1.0*i_dpi*v_td2i + 1.0*i_dpr*v_td0r + 1.0*i_dpr*v_td2r - 1.0*i_q0i*v_tqpi + 1.0*i_q0r*v_tqpr + 1.0*i_q2i*v_tqpi + 1.0*i_q2r*v_tqpr - 1.0*i_qpi*v_tq0i + 1.0*i_qpi*v_tq2i + 1.0*i_qpr*v_tq0r + 1.0*i_qpr*v_tq2r

In [6]:
V_abc = np.array([238,195,217])
np.max(V_abc)/np.sum(V_abc)/3

0.12205128205128206

In [30]:
params_dict = {'R':0.1,'L':0.01}
u_ini_dict = {'v_d0r':0.0,'v_q0r':0.0,
              'v_d2r':0.0,'v_q2r':0.0,
              'v_d0i':0.0,'v_q0i':0.0,
              'v_d2i':0.0,'v_q2i':0.0,
              'omega':2*np.pi*50}   

u_run_dict = u_ini_dict

i_d0r,i_q0r,i_d2r,i_q2r,i_d0i,i_q0i,i_d2i,i_q2i = sym.symbols('i_d0r,i_q0r,i_d2r,i_q2r,i_d0i,i_q0i,i_d2i,i_q2i', real=True)
v_d0r,v_q0r,v_d2r,v_q2r,v_d0i,v_q0i,v_d2i,v_q2i = sym.symbols('v_d0r,v_q0r,v_d2r,v_q2r,v_d0i,v_q0i,v_d2i,v_q2i', real=True)
R,L,omega = sym.symbols('R,L,omega', real=True) 

v_d0 = v_d0r + sym.I*v_d0i
v_q0 = v_q0r + sym.I*v_q0i
v_d2 = v_d2r + sym.I*v_d2i
v_q2 = v_q2r + sym.I*v_q2i

i_d0 = i_d0r + sym.I*i_d0i
i_q0 = i_q0r + sym.I*i_q0i
i_d2 = i_d2r + sym.I*i_d2i
i_q2 = i_q2r + sym.I*i_q2i

K = 0
di_d0 = (v_d0 - R * i_d0 + L*omega*i_q0 - sym.I*K*omega*L*i_d0)/L
di_q0 = (v_q0 - R * i_q0 - L*omega*i_d0 - sym.I*K*omega*L*i_q0)/L

di_d0r = sym.re(di_d0)
di_q0r = sym.re(di_q0)
di_d0i = sym.im(di_d0)
di_q0i = sym.im(di_q0)

i_q2_ = -sym.I*i_d2 
i_d2_ =  sym.I*i_q2 


K = 2
di_d2 = (v_d2 - R * i_d2 + L*omega*i_q2 - sym.I*K*omega*L*i_d2)/L
di_q2 = (v_q2 - R * i_q2 - L*omega*i_d2 - sym.I*K*omega*L*i_q2)/L

di_d2r = sym.re(di_d2)
di_q2r = sym.re(di_q2)
di_d2i = sym.im(di_d2)
di_q2i = sym.im(di_q2)

f_list = [di_d0r,di_q0r,di_d0i,di_q0i,di_d2r,di_q2r,di_d2i,di_q2i]
x_list = [ i_d0r, i_q0r, i_d0i, i_q0i, i_d2r, i_q2r, i_d2i, i_q2i]
g_list = []
y_list = []

h_dict = {'v_d0r':v_d0r,'v_q0r':v_q0r}

In [12]:
f_list

[-omega*(re(i_dni) + im(i_dnr)) + (-R*(re(i_dnr) - im(i_dni)) + re(v_dnr) - im(v_dni))/L,
 omega*(re(i_qnr) - im(i_qni)) + (-R*(re(i_qni) + im(i_qnr)) + re(v_qni) + im(v_qnr))/L,
 (-R*(re(i_d0r) - im(i_d0i)) + re(v_d0r) - im(v_d0i))/L,
 (-R*(re(i_q0i) + im(i_q0r)) + re(v_q0i) + im(v_q0r))/L,
 omega*(re(i_dpi) + im(i_dpr)) + (-R*(re(i_dpr) - im(i_dpi)) + re(v_dpr) - im(v_dpi))/L,
 -omega*(re(i_qpr) - im(i_qpi)) + (-R*(re(i_qpi) + im(i_qpr)) + re(v_qpi) + im(v_qpr))/L]

In [31]:
sys = {'name':f"rl_dp",
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':h_dict
      }

bldr = db.builder(sys)
bldr.build()

system without algebraic equations, adding dummy algebraic equation
Compilation time: 2.90 s


In [34]:
f_list

[(L*i_q0r*omega - R*i_d0r + v_d0r)/L,
 (-L*i_d0r*omega - R*i_q0r + v_q0r)/L,
 (L*i_q0i*omega - R*i_d0i + v_d0i)/L,
 (-L*i_d0i*omega - R*i_q0i + v_q0i)/L,
 (2*L*i_d2i*omega + L*i_q2r*omega - R*i_d2r + v_d2r)/L,
 (-L*i_d2r*omega + 2*L*i_q2i*omega - R*i_q2r + v_q2r)/L,
 (-2*L*i_d2r*omega + L*i_q2i*omega - R*i_d2i + v_d2i)/L,
 (-L*i_d2i*omega - 2*L*i_q2r*omega - R*i_q2i + v_q2i)/L]

In [33]:
x_list

[i_d0r, i_q0r, i_d0i, i_q0i, i_d2r, i_q2r, i_d2i, i_q2i]